# Before Using this notebook
Put a copy of a suitable [HF Auth Token](https://huggingface.co/docs/hub/security-tokens) in a file named `hf_auth` with no new line (to be read by the following code in `edict_functions.py`)
```
with open('hf_auth', 'r') as f:
    auth_token = f.readlines()[0].strip()
    
```

Example file at `./hf_auth`
```
abc123abc123
```


Also, run  `conda env create -f environment.yaml`, activate that conda env (`conda activate edict`). Run jupyter with that conda env active

In [ ]:
from edict_functions import *

# Generations

To run a novel EDICT generation, use `coupled_stablediffusion(my_prompt)`. This function also takes a steps kwarg that defaults to 50 (can be helpful for more complex generations but rarely needed).

EDICT doesn't offer new capabilities for straight-up text-to-image generation, but it's a good sanity check to see how good the generative process we're relying on is.


In [ ]:
def plot_EDICT_outputs(im_tuple):
    fig, (ax0, ax1) = plt.subplots(1, 2)
    ax0.imshow(im_tuple[0])
    ax1.imshow(im_tuple[1])
    plt.show()

In [ ]:
plot_EDICT_outputs(coupled_stablediffusion('A black bear'))

In [ ]:
plot_EDICT_outputs(coupled_stablediffusion('A statue of a horse'))

# Reconstruction

Given an image (`x0`), we can invert it into latents `(xt,yt)` and reconstruct it with EDICT passes in different directtions.

`run_baseline=True` is the DDIM-C baseline method from our paper, to run DDIM-UC `prompt` should be the empty string `''`

In [ ]:
im = load_im_into_format_from_path('experiment_images/church.jpg')
prompt = 'A church'
run_baseline = False

latents = coupled_stablediffusion(prompt,
                               reverse=True,
                                init_image=im,
                                run_baseline=run_baseline,
                               )
if run_baseline:
    latents = latents[0]
recon = coupled_stablediffusion(prompt,
                               reverse=False,
                                fixed_starting_latent=latents,
                                run_baseline=run_baseline,
                               )
recon = recon[0]

fig, (ax0, ax1) = plt.subplots(1,2)
ax0.imshow(im)
ax0.set_title("Original")
ax1.imshow(recon)
ax1.set_title("Recon")
plt.tight_layout()
plt.show()


In [ ]:
im = load_im_into_format_from_path('experiment_images/church.jpg')
prompt = 'A church'
run_baseline = True # Try vanilla DDIM

latents = coupled_stablediffusion(prompt,
                               reverse=True,
                                init_image=im,
                                run_baseline=run_baseline,
                               )
if run_baseline:
    latents = latents[0]
recon = coupled_stablediffusion(prompt,
                               reverse=False,
                                fixed_starting_latent=latents,
                                run_baseline=run_baseline,
                               )
recon = recon[0]

fig, (ax0, ax1) = plt.subplots(1,2)
ax0.imshow(im)
ax0.set_title("Original")
ax1.imshow(recon)
ax1.set_title("Recon")
plt.tight_layout()
plt.show()


# Editing


We provide a function `EDICT_editing` which accepts an image path, base prompt (original description) and desired edit prompt (target description).

In [ ]:
im_path = 'experiment_images/imagenet_cake.jpg'
base_prompt = 'A cupcake'
display(load_im_into_format_from_path(im_path))
for edit_prompt in ['An Easter cupcake',
                   'A hedgehog cupcake',
                   'An England Union Jack cupcake',
                   'A Chinese New Year cupcake',
                   'A rainbow cupcake']:
    print(edit_prompt)
    display(EDICT_editing(im_path,
              base_prompt,
              edit_prompt)[0])

In [ ]:
im_path = 'experiment_images/imagenet_camel.jpg'
display(load_im_into_format_from_path(im_path))
EDICT_editing(im_path,
              'Camel by a fence with a sign',
              'Camel by a fence',
             run_baseline=False,
             init_image_strength=0.8)[0]

Functionality to re-create the dog image edits from our paper (Figures 1 and S16-22)

In [ ]:
for i in range(1, 8):
    im_path = f'experiment_images/imagenet_dog_{i}.jpg'
    base_prompt = 'A dog' # poodle, dalmatian, lab, german shepherd
    print("Original")
    display(load_im_into_format_from_path(im_path))
    for breed in ['golden retriever', 'chihuahua', 'poodle', 'dalmatian', 'german shepherd', 'husky']:
        print(i, breed)
        edit_prompt = f'A {breed}'
        im0, im0v2 = EDICT_editing(im_path,
                  base_prompt,
                  edit_prompt,
                 run_baseline=False)
        display(im0)
        display(im0v2)

# Testing on more images

We provide many of the images displayed in the paper in the [experiment_images/](experiment_images/) folder.

We highly encourage you to test EDICT on in-the-wild images! Editing success varies across images, but generally meaningful edits can be performed. Consider for example [this wikimedia image](https://upload.wikimedia.org/wikipedia/commons/a/a6/Aurora_in_Abisko_near_Tornetr%C3%A4sk.jpg) that was the [Picture of the Day](https://commons.wikimedia.org/wiki/Commons:Picture_of_the_day) while writing this open-source notebook (Dec. 5 2022).

First we retrieve the image

In [ ]:
! mkdir web_images
! wget https://upload.wikimedia.org/wikipedia/commons/a/a6/Aurora_in_Abisko_near_Tornetr%C3%A4sk.jpg -O web_images/aurora.jpg

Let's check it out

In [ ]:
im_path = 'web_images/aurora.jpg'
display(load_im_into_format_from_path(im_path))


In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A polar bear watching a green aurora over a snowy landscape',
             run_baseline=False,
             init_image_strength=0.8)[0]

In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A red aurora over a snowy landscape',
             run_baseline=False,
             init_image_strength=0.8)[0]

In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A couple getting their photo taken in front of a green aurora over a snowy landscape',
             run_baseline=False,
             init_image_strength=0.8)[0]
# Here we notice the scenery changing
# Let's see what we can make it change to if we try

In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A green aurora over mountains', # fairly guaranteed this would work from previous edit mistake
             run_baseline=False,
             init_image_strength=0.8)[0]

In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A green aurora over a snowy lake',
             run_baseline=False,
             init_image_strength=0.8)[0]

In [ ]:
EDICT_editing(im_path,
              'A green aurora over a snowy landscape',
              'A green aurora over a forest',
             run_baseline=False,
             init_image_strength=0.8)[0]

# Happy Edi(c)ting!